In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3890
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [5]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_

In [6]:
import json

with open('/home/husein/translation/alignment.alignment') as fopen:
    for i in fopen:
        data = json.loads(i)
        break

In [7]:
data['left'][0]

'I will fight with them till they become like us."'

In [9]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 15):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l].istitle() or splitted_right[r].istitle():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [10]:
consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, right, alignment):
    
    left = random_replace_alignment(left, right, alignment)
    
    return left

In [12]:
k = 10
alignment = data['forward'][k]
left = data['left'][k]
right = data['right'][k]
augment(left, right, alignment)

'Except Iblis, he pula terhadap berlaku dari the kalangan shalih.'

In [14]:
from tqdm import tqdm

with open('en-ms-augment-v3.jsonl', 'w') as fwrite:
    with open('/home/husein/translation/alignment.alignment') as fopen:
        for i in tqdm(fopen):
            try:
                data = json.loads(i)
                result = []
                for k in range(len(data['right'])):
                    alignment = data['forward'][k]
                    left = data['left'][k]
                    right = data['right'][k]
                    try:
                        augmented = augment(left, right, alignment)
                        result.append({'left': left, 'right': right, 'augmented': augmented})
                    except:
                        pass
                fwrite.write(f'{json.dumps(result)}\n')
            except:
                pass

736635it [05:34, 2201.25it/s]
